# 05 - JSON mode

JSON mode in OpenAI's chat completion allows developers to receive structured data responses in JSON format. 
This feature is particularly useful for applications that require precise data parsing and integration with other systems. By specifying a JSON schema, developers can guide the model to generate responses that adhere to a predefined structure, ensuring consistency and ease of use when handling complex data interactions. This approach enhances the model's utility in scenarios like API development, data processing, and automated workflows, providing a more reliable and structured output.

First, we use the `import` statement to let our application know that we're going to be using the OpenAI library in our code.

In [ ]:
%pip install openai

We'll create a new `AzureOpenAI` object and pass in the API key and version and the endpoint URL to be used.

In [1]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION")
)
def print_response(system_prompt, user_prompt):
    response = client.chat.completions.create(
        model = os.getenv("AZURE_OPENAI_COMPLETION_MODEL"),
        messages = [{"role" : "system", "content" : system_prompt}, {"role" : "user", "content" : user_prompt}],
    )
    print(response.choices[0].message.content)

Found Azure OpenAI API Base Endpoint: https://aoi-sec.openai.azure.com/


## Send a prompt to Azure OpenAI using the OpenAI library

Now that we have defined an Azure OpenAI instance, let's try a Chat Completion. We'll call the `chat.completions.create()` method. Note that for the `model` value, we actually pass in the id of our Azure OpenAI `deployment`. We'll also pass the `prompt` we want to use as the `content` of the `messages` parameter.

* Exercise: Write a prompt that generates the expected completion
* Input text:
  ```
  Hello, my name is Mateo Gomez. I lost my Credit card on August 17th, and I would like to request its cancellation. The last purchase I made was of a Chicken parmigiana dish at Contoso Restaurant, located near the Hollywood Museum, for $40. Below is my personal information for validation:
  Profession: Accountant
  Social Security number is 123-45-6789
  Date of birth: 9-9-1989
  Phone number: 949-555-0110
  Personal address: 1234 Hollywood Boulevard Los Angeles CA
  Linked email account: mateo@contosorestaurant.com
  Swift code: CHASUS33XXX
  ```
* Expected completion:
  ```
  {
      "reason": "Lost card",
      "classified_reason": "lost_card",
      "name": "Mateo Gomez",
      "ssn": "123-45-6789",
      "dob": "09/09/1989"
  }
  ```

In [2]:
system_prompt ="""
Given an input text containing a customer's personal information and request details, extract and structure the relevant information into a JSON object. 
The object should include the reason for the request, a classified reason code, and key personal details of the customer. 
"""
user_prompt =""" 
  Hello, my name is Mateo Gomez. I lost my Credit card on August 17th, and I would like to request its cancellation. The last purchase I made was of a Chicken parmigiana dish at Contoso Restaurant, located near the Hollywood Museum, for $40. Below is my personal information for validation:
  Profession: Accountant
  Social Security Number is 123-45-6789
  Date of birth: 9-9-1989
  Phone number: 949-555-0110
  Personal address: 1234 Hollywood Boulevard Los Angeles CA
  Linked email account: mateo@contosorestaurant.com
  Swift code: CHASUS33XXX
  """
print_response(system_prompt=system_prompt, user_prompt=user_prompt)

```json
{
  "request": {
    "reason": "Request for cancellation of lost credit card",
    "reason_code": "CARD_LOST",
    "customer_details": {
      "name": "Mateo Gomez",
      "profession": "Accountant",
      "social_security_number": "123-45-6789",
      "date_of_birth": "1989-09-09",
      "phone_number": "949-555-0110",
      "address": "1234 Hollywood Boulevard, Los Angeles, CA",
      "email": "mateo@contosorestaurant.com"
    },
    "purchase_details": {
      "last_purchase": {
        "description": "Chicken parmigiana dish",
        "location": "Contoso Restaurant",
        "price": 40,
        "date": "2023-08-17"
      }
    }
  }
}
```


# Can we do better?

We can provide additional examples or expected completions to the model to improve accuracy. 

In [3]:
system_prompt ="""
Given an input text containing a customer's personal information and request details, extract and structure the relevant information into a JSON object. 
The object should include the reason for the request, a classified reason code, and key personal details of the customer. 

* Expected completion:

  {
      "reason": "Lost card",
      "classified_reason": "lost_card",
      "name": "Mateo Gomez",
      "ssn": "123-45-6789",
      "dob": "09/09/1989"
  }

"""
print_response(system_prompt=system_prompt, user_prompt=user_prompt)

```json
{
  "reason": "Lost card",
  "classified_reason": "lost_card",
  "name": "Mateo Gomez",
  "ssn": "123-45-6789",
  "dob": "09/09/1989"
}
```


## Can we do better than better?

Yes - with JSON mode.

JSON Mode is a feature that ensures the model outputs **valid JSON** according to a specified schema. When enabled, the model strictly adheres to JSON formatting rules, making it ideal for structured data generation, API responses, or integrations where predictable output is critical.

**Key Points:**

*   Guarantees **well-formed JSON** (no extra text or invalid syntax).
*   Works with **response\_format={"type": "json\_object"}** in the API.
*   Useful for tasks like structured data extraction, configuration generation, or passing data between systems.
*   Reduces post-processing and parsing errors compared to free-form text output.



In [4]:
def print_response(system_prompt, user_prompt):
    response = client.chat.completions.create(
        model = os.getenv("AZURE_OPENAI_COMPLETION_MODEL"),
        messages = [{"role" : "system", "content" : system_prompt}, {"role" : "user", "content" : user_prompt}],
        response_format={"type": "json_object"}

    )
    print(response.choices[0].message.content)

print_response(system_prompt=system_prompt, user_prompt=user_prompt)

{
    "reason": "Lost card",
    "classified_reason": "lost_card",
    "name": "Mateo Gomez",
    "ssn": "123-45-6789",
    "dob": "09/09/1989"
}


### Limitations of JSON Mode:

In [5]:
system_prompt ="""Extract data from the given text"""
print_response(system_prompt=system_prompt, user_prompt=user_prompt)


BadRequestError: Error code: 400 - {'error': {'message': "'messages' must contain the word 'json' in some form, to use 'response_format' of type 'json_object'.", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}

In [6]:

system_prompt ="""Extract data from the given text in JSON"""
print_response(system_prompt=system_prompt, user_prompt=user_prompt)

{
  "name": "Mateo Gomez",
  "lost_card_date": "August 17th",
  "cancellation_request": true,
  "last_purchase": {
    "item": "Chicken parmigiana",
    "location": "Contoso Restaurant",
    "address": "near the Hollywood Museum",
    "amount": 40
  },
  "personal_information": {
    "profession": "Accountant",
    "social_security_number": "123-45-6789",
    "date_of_birth": "9-9-1989",
    "phone_number": "949-555-0110",
    "address": "1234 Hollywood Boulevard, Los Angeles, CA",
    "email": "mateo@contosorestaurant.com",
    "swift_code": "CHASUS33XXX"
  }
}


# Can we do even better?

Expected completion:
  ```
  {
      "reason": "Lost card",
      "classified_reason": "lost_card",
      "name": "Mateo Gomez",
      "ssn": "123-45-6789",
      "dob": "09/09/1989"
  }
  ```



  ## Structured Outputs
  
  Structured Outputs is the evolution of JSON mode. While both ensure valid JSON is produced, only Structured Outputs ensure schema adherance. Both Structured Outputs and JSON mode are supported in the Responses API,Chat Completions API, Assistants API, Fine-tuning API and Batch API.

We recommend always using Structured Outputs instead of JSON mode when possible. This will help future-proof your integration and make it easier to work with the API.

| Feature              | Structured Outputs                                      | JSON Mode                                   |
|-----------------------|---------------------------------------------------------|--------------------------------------------|
| Outputs valid JSON    | Yes                                                     | Yes                                        |
| Adheres to schema     | Yes (see supported schemas)                             | No                                         |
| Compatible models     | gpt-4o-mini, gpt-4o-2024-08-06, and later               | gpt-3.5-turbo, gpt-4-*, and gpt-4o-* models |
| Enabling              | `text: { format: { type: "json_schema", "strict": true, "schema": ... } }` | `text: { format: { type: "json_object" } }` |

More info on [Structured Outputs](https://platform.openai.com/docs/guides/structured-outputs?example=structured-data) and [Structured Outputs Cookbook](https://cookbook.openai.com/examples/structured_outputs_intro).

In [7]:
schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "my_schema",
        "schema": {
            "type": "object",
            "properties": {
                "reason": {
                    "type": "string",
                    "description": "Reason for the request"
                },
                "classified_reason": {
                    "type": "string",
                    "description": "Classified reason code can be one of 'lost_card', 'account_closure', 'address_change' or 'unknown'"
                },
                "name": {
                    "type": "string"
                },
                "ssn": {
                    "type": "string"
                },
                "dob": {
                    "type": "string"
                }
            },
            "required": ["reason", "classified_reason", "name", "ssn", "dob"],
        }
    }
}

Use the schema to validate JSON data using structured output. 

In [8]:
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = "2024-10-21"
)

def print_response(system_prompt, user_prompt):
    response = client.chat.completions.create(
        model = os.getenv("AZURE_OPENAI_COMPLETION_MODEL"),        
        messages = [{"role" : "system", "content" : system_prompt}, {"role" : "user", "content" : user_prompt}],
        response_format=schema

    )
    print(response.choices[0].message.content)
system_prompt ="""Extract data from the given text"""
user_prompt =""" 
  Hello, my name is Mateo Gomez. I lost my Credit card on August 17th, and I would like to request its cancellation. The last purchase I made was of a Chicken parmigiana dish at Contoso Restaurant, located near the Hollywood Museum, for $40. Below is my personal information for validation:
  Profession: Accountant
  Social Security Number is 123-45-6789
  Date of birth: 9-9-1989
  Phone number: 949-555-0110
  Personal address: 1234 Hollywood Boulevard Los Angeles CA
  Linked email account: mateo@contosorestaurant.com
  Swift code: CHASUS33XXX
  """
print_response(system_prompt=system_prompt, user_prompt=user_prompt)

{"name":"Mateo Gomez","ssn":"123-45-6789","dob":"9-9-1989","reason":"I lost my Credit card on August 17th, and I would like to request its cancellation.","classified_reason":"lost_card"}


In addition to supporting JSON Schema in the REST API, the OpenAI SDKs for Python and JavaScript also make it easy to define object schemas using Pydantic and Zod respectively.
The new version of the SDK introduces a **parse** helper to provide your own Pydantic model instead of having to define the JSON schema. **We recommend using this method if possible**.

In [10]:
from pydantic import BaseModel, Field

class CustomerRequest(BaseModel):
    reason: str = Field(description="Reason for the request")
    classified_reason: str = Field(
        description="Classified reason code can be one of 'lost_card', 'account_closure', 'address_change' or 'unknown'"
    )
    name: str
    ssn: str
    dob: str

response = client.chat.completions.parse(
        model = os.getenv("AZURE_OPENAI_COMPLETION_MODEL"),        
        messages = [{"role" : "system", "content" : system_prompt}, {"role" : "user", "content" : user_prompt}],
        response_format=CustomerRequest
    )
print(response.choices[0].message.content)

{"reason":"I lost my Credit card and would like to request its cancellation.","classified_reason":"lost_card","name":"Mateo Gomez","ssn":"123-45-6789","dob":"1989-09-09"}
